Distribution Distance Measurement
===

This is a brief summery about some commonly used distance and similarity measurement in machine learning. Accurate distance measurement is a key block of many machine learning algorithms.

In [49]:
# import some necessary packages and modules

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Function

## 1. Common distance and similarity measurement

### 1.1 Euclidean distance

Defined on two vectors (two points in space) : Euclidean distance of point $\bold x$ and point $\bold y $  is:
$$d_{\text Euclidean} = \sqrt {(\bold x - \bold y)^\top (\bold x - \bold y)}

In [53]:
# Define two matrices
matrix1 = torch.tensor([[1, 2, 3], [4, 5, 6], [7, 8, 9]], dtype=float)
matrix2 = torch.tensor([[9, 8, 7], [6, 5, 4], [3, 2, 1]], dtype=float)

# Compute the Euclidean distance
euclidean_distance = torch.norm(matrix1 - matrix2, p=2)

print('Euclidean distance:', euclidean_distance.item())

Euclidean distance: 15.491933384829668


### 1.2 Minkowski distance

Defined on two vectors (two points in space) : $p$-order Minkowski distance of point $\bold x$ and point $\bold y $  is:
$$d_{\text Minkowski} = (||\bold x - \bold y||^p)^{\frac {i}{p}}$$
when $p=1$, it is Manhattan distance; $p=2$, it is Euclidean distance.

### 1.3 Mahalanobis distance

Defined on two vectors (two points in space), and these two points are in the same distribution: Mahalanobis distance of point $\bold x$ and point $\bold y $  is:
$$d_{Mahalanobis}=\sqrt {(\bold x - \bold y)^\top \textstyle \sum ^{-1}(\bold x - \bold y)}$$
where $\sum$ is the covariance matrix of this distribution. 
When $\sum = \bold I$, it is Euclidean distance.

## 2. Cosine similarity

Measure the correlation of two vectors (cosine of the Angle between them): Cosine similarity of vector $\bold x$ and vector $\bold y $ is:
$$\text cos(\bold x, \bold y=\frac {\bold x \cdot \bold y}{|\bold x| \cdot |\bold y|})$$

Cosine distance is widely used in time series data to measure the similarity between two distributions. Given the data $\bold h_s$ and $\bold h_t$ , cosine distance is defined as:
$$d_{cosine}(\bold h_s, \bold h_t)=1-\frac {\langle \bold h_s, \bold h_t\rangle}{||\bold h_s||\cdot||\bold h_t||}$$

nn.CosineSimilarity()
$$ \text{similarity} = \dfrac{x_1 \cdot x_2}{\max(\Vert x_1 \Vert _2 \cdot \Vert x_2 \Vert _2, \epsilon)}$$

In [35]:
# nn.CosineSimilarity()
def cosine(source, target):
    # source, target = source.mean(0), target.mean(0)
    cos = nn.CosineSimilarity(dim=0)
    loss = cos(source, target)
    return loss.mean()

## 3. Mutual information

Defined on two distributions $X$ and $Y$, mutual information of them is:
$$I(X,Y)=\sum_{x \in X}\sum_{y \in Y}p(x,y)log \frac {p(x,y)}{p(x)p(y)}$$

## 4. Correlation coefficient

### 4.1 Pearson's correlation coefficient

Measure the correlation between two random variables $X$ and $Y$：
$$\rho _{X,Y}=\frac {Cov(X,Y)}{\sigma _X \sigma _Y}$$
where $Cov(X,Y)$ denotes the covariance matrix, $\sigma$ is the standard deviation.
Pearson's correlation coefficient ranges [-1, 1], and a greater absolute value indicates a bigger correlation.

### 4.2 Jaccard correlation coefficient

Measure the correlation between two sets $X$ and $Y$：
$$J=\frac{X \cap Y}{X \cup Y}$$
Jaccard distance:
$$d_{\text Jaccard}=1-J$$

## 5. Kullback-Leibler divergence and Jensen-Shannon Divergence

**Kullback-Leibler divergence** measures the distance between two probability distributions $P(x)$ and $Q(x)$:
$$D_{KL}(P||Q)=\sum_{i=1}P(x)log\frac {P(x)}{Q(x)}$$
This is an asymmetric distance: $D_{KL}(P||Q) \neq D_{KL}(Q||P)$

The basic idea is to compare the difference in probability between two distributions on the same event. The smaller the KL divergence, the more similar the two distributions are.

`torch.nn.KLDivloss()`: The Kullback-Leibler divergence loss.

For tensors of the same shape $y_{\text{pred}}$, $y_{\text{true}}$, where $y_{\text{pred}}$ is the `input` and $y_{\text{true}}$ is the `target`, we define the **pointwise KL-divergence** as
$$L(y_{\text{pred}},\ y_{\text{true}})
    = y_{\text{true}} \cdot \log \frac{y_{\text{true}}}{y_{\text{pred}}}
    = y_{\text{true}} \cdot (\log y_{\text{true}} - \log y_{\text{pred}})$$
To avoid underflow issues when computing this quantity, this loss expects the argument `input` in the log-space.

**Jensen-Shannon Divergence** is the mean of KL divergence, and it is symmetric and non-negative. The smaller the JS divergence, the more similar the two distributions are.
$$JSP(P||Q)=\frac {1}{2}D_{KL}(P||M)+\frac {1}{2}D_{KL}(Q||M)$$
where $M=\frac {1}{2}(P+Q)$.


In [36]:
# nn.KLDivLoss()
def kl_div(source, target):
    if len(source) < len(target):
        target = target[:len(source)]
    elif len(source) > len(target):
        source = source[:len(target)]
    criterion = nn.KLDivLoss(reduction='batchmean')
    loss = criterion(source.log(), target)
    return loss

def js(source, target):
    if len(source) < len(target):
        target = target[:len(source)]
    elif len(source) > len(target):
        source = source[:len(target)]
    M = .5 * (source + target)
    loss_1, loss_2 = kl_div(source, M), kl_div(target, M)
    return .5 * (loss_1 + loss_2)

## 6. Maximum Mean Discrepancy(MMD)

The MMD is a non-parametric distance between two probability distributions. MMD measures the distance of the mean embeddings of the samples from two distributions in a Reproducing Kernel Hilbert Space (RKHS). MMD can be empirically estimated by
$$d_{mmd}(\bold h_s, \bold h_t)=\frac {1}{n_s^2}\sum^{n_s}_{i,j=1}k(h_{s_i},h_{s_j})+\frac {1}{n_t^2}\sum_{i,j=n_s+1}^{n_s+n_t}k(h_{t_i},h_{t_j})-\frac {2}{n_s n_t}\sum_{i=1}^{n_s}\sum_{j=n_s+1}^{n_s+n_t}k(h_{s_i},h_{t_j})\sum_{i=n_s+1}^{n_s+n_t}\sum_{j=1}^{n_s}k(h_{s_i},h_{t_j})$$
where $k(\cdot,\cdot)$ is the kernel function such as RBF kernel and linear kernel, and $n_s=|h_s|$, $n_t=|h_t|$ are the number of the data from two distributions.

In [37]:
class MMD_loss(nn.Module):
    def __init__(self, kernel_type='linear', kernel_mul=2.0, kernel_num=5):
        super(MMD_loss, self).__init__()
        self.kernel_num = kernel_num
        self.kernel_mul = kernel_mul
        self.fix_sigma = None
        self.kernel_type = kernel_type

    def guassian_kernel(self, source, target, kernel_mul=2.0, kernel_num=5, fix_sigma=None):
        n_samples = int(source.size()[0]) + int(target.size()[0])   # n_s + n_t
        total = torch.cat([source, target], dim=0)  # (n_samples, d)
        total0 = total.unsqueeze(0).expand(
            int(total.size(0)), int(total.size(0)), int(total.size(1)))     # (1, n_samples, d) -> (n_samples, n_samples, d)
        total1 = total.unsqueeze(1).expand(
            int(total.size(0)), int(total.size(0)), int(total.size(1)))     # (n_samples, 1, d) -> (n_samples, n_samples, d)
        L2_distance = ((total0-total1)**2).sum(2)       # (n_samples, n_samples)    (x_i - x_j)**2
        if fix_sigma:
            bandwidth = fix_sigma
        else:
            bandwidth = torch.sum(L2_distance.data) / (n_samples**2-n_samples)
        bandwidth /= kernel_mul ** (kernel_num // 2)
        bandwidth_list = [bandwidth * (kernel_mul**i)
                          for i in range(kernel_num)]        # len() = n_samples    
        kernel_val = [torch.exp(-L2_distance / bandwidth_temp)
                      for bandwidth_temp in bandwidth_list]     # len() = n_samples  kernel_val[i].shape:(n_samples, n_samples)
        return sum(kernel_val)      # (n_samples, n_samples)

    def linear_mmd(self, X, Y):
        delta = X.mean(axis=0) - Y.mean(axis=0)
        loss = delta.dot(delta.T)
        return loss

    def forward(self, source, target):
        if self.kernel_type == 'linear':
            return self.linear_mmd(source, target)
        elif self.kernel_type == 'rbf':
            batch_size = int(source.size()[0])  # n_s
            kernels = self.guassian_kernel(
                source, target, kernel_mul=self.kernel_mul, kernel_num=self.kernel_num, fix_sigma=self.fix_sigma)
            with torch.no_grad():
                XX = torch.mean(kernels[:batch_size, :batch_size])
                YY = torch.mean(kernels[batch_size:, batch_size:])
                XY = torch.mean(kernels[:batch_size, batch_size:])
                YX = torch.mean(kernels[batch_size:, :batch_size])
                loss = torch.mean(XX + YY - XY - YX)
            return loss

def mmd(source, target, kernel_type='linear', kernel_mul=2.0, kernel_num=5):
    model = MMD_loss(kernel_type, kernel_mul, kernel_num)
    return model(source, target)

## Domain Adversarial Discrepancy

The domain discrepancy can be parameterized as a neural network and it is referred to as the domain adversarial discrepancy[1]. An additional network named domain discriminator denoted by 𝐷 is introduced. The domain adversarial objective is defined as: 

$$l_{adv}(\bold h_s, \bold h_t) = \mathbb E[log[D(\bold h_s)]] + \mathbb E[log[1-D(\bold h_t)]]$$

where $\mathbb E$ denotes expectation. Hence, the adversarial discrepancy is 
$$d_{adv}(\bold h_s, \bold h_t) = -l_{adv}(\bold h_s, \bold h_t)$$


`[1] Yaroslav Ganin, E. Ustinova, Hana Ajakan, P. Germain, H. Larochelle, F. Laviolette, M. Marchand, and V. Lempitsky. 2016. Domain-Adversarial Training of Neural Networks. JMLR 17 (2016), 59:1–59:35`

In [38]:
class Discriminator(nn.Module):
    '''two layer net: affine - > relu - > affine - > sigmoid'''
    def __init__(self, input_dim=256, hidden_dim=256):
        super(Discriminator, self).__init__()
        self.input_dim = input_dim
        self.hidden_dim = hidden_dim
        self.dis1 = nn.Linear(input_dim, hidden_dim)
        self.dis2 = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        x = F.relu(self.dis1(x))
        x = self.dis2(x)
        x = torch.sigmoid(x)
        return x


class ReverseLayerF(Function):

    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha
        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha
        return output, None

# nn.BCELoss()
def adv(source, target, input_dim=256, hidden_dim=512):
    domain_loss = nn.BCELoss()
    adv_net = Discriminator(input_dim, hidden_dim).cuda()
    domain_src = torch.ones((len(source), 1)).cuda()     # labels of source domain 1
    domain_tar = torch.zeros((len(target), 1)).cuda()    # labels of target domain 0
    reverse_src = ReverseLayerF.apply(source, 1)        # ???
    reverse_tar = ReverseLayerF.apply(target, 1)
    pred_src = adv_net(reverse_src)
    pred_tar = adv_net(reverse_tar)
    loss_s, loss_t = domain_loss(
        pred_src, domain_src), domain_loss(pred_tar, domain_tar)
    loss = loss_s + loss_t
    return loss

## Deep Correlation Alignment(CORAL)

The CORAL distance is defined as the distance of the second-order statistic (covariance) of the samples from two distributions:
$$d_{coral}(\bold h_s, \bold h_t) = \frac {1}{4q^2}||\bold C_s-C_t||_F^2$$

where $q$ is the dimension of the two distributions.

In [39]:
def CORAL(source, target):
    d = source.size(1)  # feature dimension
    ns, nt = source.size(0), target.size(0)     # number of samples

    # source covariance
    tmp_s = torch.ones((1, ns)).cuda() @ source     # torch.Size: ([1,d])
    cs = (source.t() @ source - (tmp_s.t() @ tmp_s) / ns) / (ns - 1)    # torch.Size: ([d,d])

    # target covariance
    tmp_t = torch.ones((1, nt)).cuda() @ target
    ct = (target.t() @ target - (tmp_t.t() @ tmp_t) / nt) / (nt - 1)

    # frobenius norm
    loss = (cs - ct).pow(2).sum()
    loss = loss / (4 * d * d)

    return loss

In [45]:
class Mine(nn.Module):
    def __init__(self, input_dim=2048, hidden_dim=512):
        super(Mine, self).__init__()
        self.fc1_x = nn.Linear(input_dim, hidden_dim)
        self.fc1_y = nn.Linear(input_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, 1)

    def forward(self, x, y):
        h1 = F.leaky_relu(self.fc1_x(x)+self.fc1_y(y))
        h2 = self.fc2(h1)
        return h2


class Mine_estimator(nn.Module):
    def __init__(self, input_dim=2048, hidden_dim=512):
        super(Mine_estimator, self).__init__()
        self.mine_model = Mine(input_dim, hidden_dim)

    def forward(self, X, Y):
        Y_shuffle = Y[torch.randperm(len(Y))]
        loss_joint = self.mine_model(X, Y)
        loss_marginal = self.mine_model(X, Y_shuffle)
        ret = torch.mean(loss_joint) - \
            torch.log(torch.mean(torch.exp(loss_marginal)))
        loss = ret if ret == 0 else -ret
        return loss

def mine(source, target, input_dim=2048, hidden_dim=512):
    model = Mine_estimator(input_dim, hidden_dim).cuda()
    return model(source, target)



In [47]:
def pairwise_dist(X, Y):
    n, d = X.shape
    m, _ = Y.shape
    assert d == Y.shape[1]
    a = X.unsqueeze(1).expand(n, m, d)
    b = Y.unsqueeze(0).expand(n, m, d)
    return torch.pow(a - b, 2).sum((0, 1, 2))


def pairwise_dist_np(X, Y):
    n, d = X.shape
    m, _ = Y.shape
    assert d == Y.shape[1]
    a = np.expand_dims(X, 1)
    b = np.expand_dims(Y, 0)
    a = np.tile(a, (1, m, 1))
    b = np.tile(b, (n, 1, 1))
    return np.power(a - b, 2).sum((0, 1, 2))


def pa(X, Y):
    XY = np.dot(X, Y.T)
    XX = np.sum(np.square(X), axis=1)
    XX = np.transpose([XX])
    YY = np.sum(np.square(Y), axis=1)
    dist = XX + YY - 2 * XY

    return dist.sum()


In [46]:
torch.manual_seed(124)
source = torch.rand((2,5),device='cuda')
target = torch.rand((2,5),device='cuda')
# source = torch.tensor([[1.,2.,3.,4.,5.],[6.,7.,8.,9.,10.]], device='cuda')
# target = torch.tensor([[1.,2.,3.,4.,5.],[6.,7.,8.,9.,10.]], device='cuda')
loss_adv = adv(source,target, 5, 16)
loss_coral = CORAL(source, target)
loss_cosine = cosine(source, target)
loss_js = js(source, target)
loss_kl = kl_div(source, target)
loss_mmd = mmd(source, target, 'rbf')
loss_mine = mine(source, target, 5, 10)
loss_pa = pairwise_dist(source, target)